<a href="https://colab.research.google.com/github/juanmed/moa-challenge-micro2025/blob/main/timm_experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install onnx mmcv mmpretrain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 479.1/479.1 kB 8.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 96.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.7/452.7 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.2/256.2 kB 26.1 MB/s eta 0:00:00
  Created wheel for mmcv: filename=mmcv-2.2.0-cp312-cp312-linux_x86_64.whl size=39940232 sha256=409448e109e61944c8a3d8d6d43f0206889ac1730ef503ec374794c79f1c3b39
  Stored in directory: /root/.cache/pip/wheels/c7/bf/8d/f3122496099a7c3bf81da99fc6a800e6abe3971f65dce706c0
Successfully built mmcv


In [15]:
import sys

import matplotlib.pyplot as plt
import PIL
from PIL import Image
import json

import torch
from torch import nn
import torchvision.transforms as T

from timm import create_model
from timm.data.constants import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD


In [26]:
NORMALIZE_MEAN = IMAGENET_DEFAULT_MEAN
NORMALIZE_STD = IMAGENET_DEFAULT_STD
SIZE = 224
TEST_IMAGE = "6.jpeg"
MODEL_NAME = "convnextv2_large.fcmae_ft_in22k_in1k_384"
#MODEL_NAME = "convnextv2_base.fcmae_ft_in22k_in1k_384"
#MODEL_NAME = "convnextv2_large.fcmae_ft_in22k_in1k_384"
EXPORT_PATH = f"{MODEL_NAME}.onnx"

In [27]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("device = ", device)
# create a ConvNeXt model : https://github.com/rwightman/pytorch-image-models/blob/master/timm/models/convnext.py
model = create_model(MODEL_NAME, pretrained=True).to(device)

model.to(device)
model.eval()

# Here we resize smaller edge to 256, no center cropping
transforms = [
              T.Resize(SIZE, interpolation=T.InterpolationMode.BICUBIC),
              T.ToTensor(),
              T.Normalize(NORMALIZE_MEAN, NORMALIZE_STD),
              ]
transforms = T.Compose(transforms)

img = PIL.Image.open(TEST_IMAGE)
img_tensor = transforms(img).unsqueeze(0).to(device)
print(img_tensor.size())

output = torch.softmax(model(img_tensor), dim=1)
top5 = torch.topk(output, k=5)
top5_prob = top5.values[0]
top5_indices = top5.indices[0]

imagenet_labels = json.load(open('label_to_words.json'))
for i in range(5):
    labels = imagenet_labels[str(int(top5_indices[i]))]
    prob = "{:.2f}%".format(float(top5_prob[i])*100)
    print(labels, prob)


device =  cuda
torch.Size([1, 3, 224, 224])
mosquitofish, Gambusia affinis 81.77%
fly-fishing 10.16%
platy, Platypoecilus maculatus 0.28%
plasmodium, Plasmodium vivax, malaria parasite 0.12%
sporozoite 0.09%


In [25]:
c = 3
h = SIZE
w = SIZE
dummy_input = torch.rand(1, c, h, w, device=device)

print("4. Exporting to onnx: ...")
torch.onnx.export(
    model,
    dummy_input,
    EXPORT_PATH,
    export_params=True,
    opset_version=12,
    training=torch.onnx.TrainingMode.EVAL,
    do_constant_folding=True,
    dynamic_axes=None
)
print(f"ONNX export finished!: {EXPORT_PATH}")


4. Exporting to onnx: ...


/tmp/ipython-input-2549276570.py:7: DeprecationWarning: You are using the legacy TorchScript-based ONNX export. Starting in PyTorch 2.9, the new torch.export-based ONNX exporter will be the default. To switch now, set dynamo=True in torch.onnx.export. This new exporter supports features like exporting LLMs with DynamicCache. We encourage you to try it and share feedback to help improve the experience. Learn more about the new export logic: https://pytorch.org/docs/stable/onnx_dynamo.html. For exporting control flow: https://pytorch.org/tutorials/beginner/onnx/export_control_flow_model_to_onnx_tutorial.html.
  torch.onnx.export(


ONNX export finished!: convnextv2_large.fcmae_ft_in22k_in1k_384.onnx


In [10]:
import torch
import mmpretrain as mmp
from mmpretrain import get_model
from mmpretrain import inference_model

model = get_model('convnext-v2-large_fcmae-in21k-pre_3rdparty_in1k')
inputs = torch.rand(1, 3, 224, 224)
out = model(inputs)
print(type(out))
# To extract features.
feats = model.extract_feat(inputs)
print(type(feats))

TypeError: NoneType takes no arguments

In [14]:
predict = inference_model('wide-resnet101_3rdparty_8xb32_in1k', 'demo/bird.JPEG')
print(predict['pred_class'])
print(predict['pred_score'])

TypeError: NoneType takes no arguments

In [13]:
mmp.list_models()

['barlowtwins_resnet50_8xb256-coslr-300e_in1k',
 'beit-base-p16_beit-in21k-pre_3rdparty_in1k',
 'beit-base-p16_beit-pre_8xb128-coslr-100e_in1k',
 'beit-base-p16_beitv2-in21k-pre_3rdparty_in1k',
 'beit-base-p16_beitv2-pre_8xb128-coslr-100e_in1k',
 'beit-base-p16_cae-pre_8xb128-coslr-100e_in1k',
 'beit-g-p14_3rdparty-eva_30m',
 'beit-g-p14_eva-30m-in21k-pre_3rdparty_in1k-336px',
 'beit-g-p14_eva-30m-in21k-pre_3rdparty_in1k-560px',
 'beit-g-p14_eva-30m-pre_3rdparty_in21k',
 'beit-g-p16_3rdparty-eva_30m',
 'beit-l-p14_3rdparty-eva_in21k',
 'beit-l-p14_eva-in21k-pre_3rdparty_in1k-196px',
 'beit-l-p14_eva-in21k-pre_3rdparty_in1k-336px',
 'beit-l-p14_eva-pre_3rdparty_in1k-196px',
 'beit-l-p14_eva-pre_3rdparty_in1k-336px',
 'beit-l-p14_eva-pre_3rdparty_in21k',
 'beit_beit-base-p16_8xb256-amp-coslr-300e_in1k',
 'beitv2_beit-base-p16_8xb256-amp-coslr-300e_in1k',
 'blip-base_3rdparty_caption',
 'blip-base_3rdparty_nlvr',
 'blip-base_3rdparty_retrieval',
 'blip-base_3rdparty_vqa',
 'blip-base_8xb1